# Notebook 1: MED-PC Extracting the Recording Data and Metadata

Brief 1-2 sentence description of notebook.

In [146]:
import os
import sys
import glob
import re
from collections import defaultdict

In [1]:
# Imports of all used packages and libraries
import pandas as pd

In [9]:
# setting path
sys.path.append('../../src')

In [61]:
# All the libraries that were created for this repository
import extract.metadata
import extract.dataframe

## Inputs & Data

Explanation of each input and where it comes from.

In [ ]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
INPUT_VARIABLE = 1
CVD_DF = pd.read_csv("CVD_cleaned.csv")
OUTPUT_DIR = r"/root/work/" # where data is saved should always be shown in the inputs

In [11]:
DATE_BINS = ["2022-04-01", "2022-06-30", "2022-08-15", "2022-10-15"]
# Convert to DatetimeIndex
DATE_BINS = pd.DatetimeIndex(DATE_BINS)

In [12]:
# Getting all the file paths of the recording files(that happen to all end in `.txt`)
ALL_MED_PC_FILE = glob.glob("./data/*.txt")

In [134]:
COHORT_TO_CAGE_TO_STRAIN = {"1": {"1": "C57", "2": "C57", "3": "CD1", "4": "CD1"},
"2": {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"},
"3": {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"},}

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Processing

- Example of what the MED-PC Recording file looks like

In [16]:
with open(ALL_MED_PC_FILE[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-05-17_11h06m_Subject 2.4.txt







Start Date: 05/17/22

End Date: 05/17/22

Subject: 2.4

Experiment: Pilot of Pilot

Group: Cage 2

Box: 3

Start Time: 11:06:01

End Time: 12:10:51

MSN: levelNP_CS_reward_laserepochON1st_noshock

A:    2699.000

D:    9000.000

F:    2000.000

G:       0.000

H:       0.000

I:       0.000

L:       0.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [20]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=ALL_MED_PC_FILE)

In [21]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/2022-05-17_11h06m_Subject 2.4.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-05-17_11h06m_Subject 2.4.txt', 'Start Date': '05/17/22', 'End Date': '05/17/22', 'Subject': '2.4', 'Experiment': 'Pilot of Pilot', 'Group': 'Cage 2', 'Box': '3', 'Start Time': '11:06:01', 'End Time': '12:10:51', 'MSN': 'levelNP_CS_reward_laserepochON1st_noshock'}


## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [22]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

- Removing all rows that don't have a cage number (which is called group in here)

In [25]:
metadata_df = metadata_df.dropna(subset="Group")

In [26]:
metadata_df = metadata_df[metadata_df["Group"] != ""]

- Getting the numbers out of the column that contains the cage information

In [30]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: re.findall(r'\d+', x)[0])

In [31]:
metadata_df["cage"].unique()

array(['2', '3', '4', '6', '1', '5'], dtype=object)

- Labeling the cohort name based on the date

In [35]:
# Turning the string into a datetime object
metadata_df["date"] = pd.to_datetime(metadata_df["Start Date"])

In [38]:
DATE_BINS

DatetimeIndex(['2022-04-01', '2022-06-30', '2022-08-15', '2022-10-15'], dtype='datetime64[ns]', freq=None)

In [104]:
cohort_labels = [str(i) for i in range(1, len(DATE_BINS))]


In [105]:
cohort_labels

['1', '2', '3']

- Grouping the cohorts

In [106]:
metadata_df['cohort'] = pd.cut(metadata_df['date'], bins=DATE_BINS, labels=cohort_labels, right=False)


In [107]:
metadata_df = metadata_df.sort_values(["cohort", "date", "Start Time", "Subject"]).reset_index(drop=True)

In [108]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,date,cohort
0,./data/2022-05-03_10h38m_Subject 1.1.txt,C:\MED-PC\Data\2022-05-03_10h38m_Subject 1.1.txt,05/03/22,05/03/22,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
1,./data/2022-05-03_10h38m_Subject 2.2.txt,C:\MED-PC\Data\2022-05-03_10h38m_Subject 2.2.txt,05/03/22,05/03/22,2.2,Pilot of Pilot,Cage1,2,10:38:27,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
2,./data/2022-05-03_10h38m_Subject 2.3.txt,C:\MED-PC\Data\2022-05-03_10h38m_Subject 2.3.txt,05/03/22,05/03/22,2.3,Pilot of Pilot,Cage1,3,10:38:37,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
3,./data/2022-05-03_10h38m_Subject 1.4.txt,C:\MED-PC\Data\2022-05-03_10h38m_Subject 1.4.txt,05/03/22,05/03/22,1.4,Pilot of Pilot,Cage1,4,10:38:47,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
4,./data/2022-05-03_11h42m_Subject 1.1.txt,C:\MED-PC\Data\2022-05-03_11h42m_Subject 1.1.txt,05/03/22,05/03/22,1.1,Pilot of Pilot,Cage 1,1,11:42:09,11:47:02,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1


In [66]:
concatted_medpc_df = pd.read_csv("./proc/concatted_medpc.csv", index_col=0)

/tmp/ipykernel_136262/254952528.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  concatted_medpc_df = pd.read_csv("./proc/concatted_medpc.csv", index_col=0)


In [71]:
concatted_medpc_df = concatted_medpc_df.drop(columns=["date"])

- Combining the recording and the metadata into one dataframe

In [109]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index', how="inner")

In [110]:
recording_and_metadata_df = recording_and_metadata_df.sort_values(["cohort", "date", "Start Time", "Subject"]).reset_index(drop=True)

In [111]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,date,cohort
0,164.42,64.0,399.0,0.0,60.01,165.33,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
1,168.00,234.0,399.0,0.0,140.01,168.10,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
2,168.17,314.0,399.0,0.0,230.01,169.28,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
3,198.39,489.0,399.0,0.0,310.01,198.73,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1
4,242.24,674.0,399.0,0.0,385.01,242.59,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1.1,Pilot of Pilot,Cage1,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1


In [112]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,date,cohort
810971,3647.59,NaN,NaN,NaN,NaN,3647.97,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4.4,CD1_vs_C57_Comparison,Cage_4,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3
810972,3648.01,NaN,NaN,NaN,NaN,3648.08,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4.4,CD1_vs_C57_Comparison,Cage_4,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3
810973,3648.14,NaN,NaN,NaN,NaN,3648.26,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4.4,CD1_vs_C57_Comparison,Cage_4,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3
810974,3648.28,NaN,NaN,NaN,NaN,3648.62,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4.4,CD1_vs_C57_Comparison,Cage_4,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3
810975,3648.64,NaN,NaN,NaN,NaN,3648.66,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4.4,CD1_vs_C57_Comparison,Cage_4,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3


## Adding other useful information

- Adding the date as the number of subsequent days since the start

In [113]:
recording_and_metadata_df["cohort"] = recording_and_metadata_df["cohort"].apply(lambda x: str(x).strip().lower()) 
recording_and_metadata_df["cage"] = recording_and_metadata_df["cage"].apply(lambda x: str(x).strip().lower()) 
recording_and_metadata_df["Subject"] = recording_and_metadata_df["Subject"].apply(lambda x: str(x).strip().lower()) 

In [114]:
recording_and_metadata_df["cohort_cage_id"] = recording_and_metadata_df.apply(lambda x: "_".join([x["cohort"], x["cage"], x["Subject"]]), axis=1)

In [115]:
recording_and_metadata_df["cohort_cage_id"].unique()

array(['1_1_1.1', '1_1_2.2', '1_1_2.3', '1_1_1.4', '1_2_2.1', '1_2_1.2',
       '1_2_1.3', '1_2_2.4', '1_3_4.1 (1)', '1_3_3.2 (2)', '1_3_4.2 (3)',
       '1_3_3.3 (4)', '1_4_3.1 (1)', '1_4_3.4 (2)', '1_4_4.3 (3)',
       '1_4_4.4 (4)', '2_1_1', '2_1_2', '2_1_3', '2_1_4', '2_2_1',
       '2_2_2', '2_2_3', '2_2_4', '2_3_1', '2_3_2', '2_3_3', '2_3_4',
       '2_4_1', '2_4_2', '2_4_3', '2_4_4', '2_5_1', '2_5_2', '2_5_3',
       '2_5_4', '2_6_1', '2_6_2', '2_6_3', '2_6_4', '3_4_4.1', '3_4_4.2',
       '3_4_4.3', '3_4_4.4', '3_1_1.1', '3_1_1.2', '3_1_1.3', '3_1_1.4',
       '3_5_5.1', '3_5_5.2', '3_5_5.3', '3_5_5.4', '3_2_2.1', '3_2_2.2',
       '3_2_2.3', '3_2_2.4', '3_6_6.1', '3_6_6.2', '3_6_6.3', '3_6_6.4',
       '3_3_3.1', '3_3_3.2', '3_3_3.3', '3_3_3.4', '3_2_5.1', '3_2_5.2',
       '3_2_5.3', '3_2_5.4'], dtype=object)

In [121]:
recording_and_metadata_df["date_str"] = recording_and_metadata_df["date"].dt.strftime('%Y-%m-%d')

In [122]:
cohort_cage_id_to_date_to_session = defaultdict(dict)

for cohort_cage_id in recording_and_metadata_df["cohort_cage_id"].unique():
    cohort_cage_id_df = recording_and_metadata_df[recording_and_metadata_df["cohort_cage_id"] == cohort_cage_id]
    for index, date in enumerate(sorted(cohort_cage_id_df["date_str"].unique())):
        cohort_cage_id_to_date_to_session[cohort_cage_id][date] = index + 1

In [123]:
date

'2022-09-26'

In [127]:
recording_and_metadata_df["session"] = recording_and_metadata_df.apply(lambda x: cohort_cage_id_to_date_to_session[x["cohort_cage_id"]][x["date_str"]], axis=1)

In [128]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Box,Start Time,End Time,MSN,cage,date,cohort,cohort_cage_id,date_str,session
0,164.42,64.0,399.0,0.0,60.01,165.33,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1
1,168.00,234.0,399.0,0.0,140.01,168.10,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1
2,168.17,314.0,399.0,0.0,230.01,169.28,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1
3,198.39,489.0,399.0,0.0,310.01,198.73,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1
4,242.24,674.0,399.0,0.0,385.01,242.59,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,1,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1


In [129]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Box,Start Time,End Time,MSN,cage,date,cohort,cohort_cage_id,date_str,session
810971,3647.59,NaN,NaN,NaN,NaN,3647.97,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8
810972,3648.01,NaN,NaN,NaN,NaN,3648.08,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8
810973,3648.14,NaN,NaN,NaN,NaN,3648.26,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8
810974,3648.28,NaN,NaN,NaN,NaN,3648.62,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8
810975,3648.64,NaN,NaN,NaN,NaN,3648.66,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,4,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8


In [61]:
recording_and_metadata_df.groupby(["subject", "date", "session"]).count().head(n=50)

(P)Portentry  (Q)USdelivery  (R)UStime  \
subject date     session                                           
1.1     20220503 1                  82             11         11   
        20220504 2                 920             41         41   
        20220505 3                 619             40         40   
        20220506 4                 447             41         41   
        20220507 5                 703             40         40   
        20220508 6                 446             39         39   
        20220509 7                 586             40         40   
        20220510 8                1278             80         80   
        20220511 9                 756             40         40   
        20220516 10                746             41         41   
        20220517 11                424             33         33   
1.2     20220503 1                  68             14         14   
        20220504 2                 364             39         39   
        20220505 3                 548             41         41   
        20220506 4                 412             40         40   
        20220507 5                 464             40         40   
        20220508 6                 471             39         39   
        20220509 7                 611             40         40   
        20220510 8                 441             40         40   
        20220511 9                 631             40         40   
        20220512 10                471             40         40   
        20220516 11                634             40         40   
        20220517 12                686             40         40   
1.3     20220503 1                  24             10         10   
        20220504 2                 195             37         37   
        20220505 3                 112             15         15   
        20220506 4                 366             39         39   
        20220507 5                 503             40         40   
        20220508 6                 536             39         39   
        20220509 7                 464             40         40   
        20220510 8                 498             40         40   
        20220511 9                 749             40         40   
        20220512 10                973             40         40   
        20220516 11               1084             40         40   
        20220517 12                753             40         40   
1.4     20220503 1                  68             11         11   
        20220504 2                 378             41         41   
        20220505 3                   5              3          3   
        20220506 4                 331             38         38   
        20220507 5                 370             39         39   
        20220508 6                 618             39         39   
        20220509 7                 353             40         40   
        20220510 8                1078             80         80   
        20220511 9                 566             39         39   
        20220516 10                642             41         41   
        20220517 11                923             40         40   
2.1     20220503 1                 202             15         15   
        20220504 2                 699             40         40   
        20220505 3                 467             38         38   
        20220506 4                 543             40         40   

                          (W)ITIvalues  (S)CSpresentation  (N)Portexit  \
subject date     session                                                 
1.1     20220503 1                2501                 74           82   
        20220504 2                2501                100          920   
        20220505 3                2501                 99          618   
        20220506 4                2501                100          446   
        20220507 5                2501       

## Adding the strain information

In [136]:
recording_and_metadata_df["strain"] = recording_and_metadata_df.apply(lambda row: COHORT_TO_CAGE_TO_STRAIN.get(row['cohort'], {}).get(row['cage'], 'No Match'), axis=1)


In [137]:
recording_and_metadata_df["strain"].unique()

array(['C57', 'CD1'], dtype=object)

In [140]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Start Time,End Time,MSN,cage,date,cohort,cohort_cage_id,date_str,session,strain
0,164.42,64.0,399.0,0.0,60.01,165.33,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1,C57
1,168.00,234.0,399.0,0.0,140.01,168.10,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1,C57
2,168.17,314.0,399.0,0.0,230.01,169.28,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1,C57
3,198.39,489.0,399.0,0.0,310.01,198.73,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1,C57
4,242.24,674.0,399.0,0.0,385.01,242.59,1.0,0.0,1.1,./data/2022-05-03_10h38m_Subject 1.1.txt,...,10:38:08,11:17:00,levelNP_CS_reward_laserepochON1st_noshock,1,2022-05-03,1,1_1_1.1,2022-05-03,1,C57


In [141]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,subject,file_path,...,Start Time,End Time,MSN,cage,date,cohort,cohort_cage_id,date_str,session,strain
810971,3647.59,NaN,NaN,NaN,NaN,3647.97,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8,CD1
810972,3648.01,NaN,NaN,NaN,NaN,3648.08,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8,CD1
810973,3648.14,NaN,NaN,NaN,NaN,3648.26,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8,CD1
810974,3648.28,NaN,NaN,NaN,NaN,3648.62,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8,CD1
810975,3648.64,NaN,NaN,NaN,NaN,3648.66,NaN,NaN,4.4,./data/2022-09-27_17h40m_Subject 4.4.txt,...,17:40:46,18:42:29,CD1_reward_training,4,2022-09-27,3,3_4_4.4,2022-09-27,8,CD1


## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [142]:
output_directory = "./proc/extracted_recording_data_and_metadata"

In [143]:
output_directory

'./proc/extracted_recording_data_and_metadata'

In [147]:
os.makedirs(output_directory, exist_ok=True)

In [149]:
metadata_df.to_csv(os.path.join(output_directory, "domcomp_pilot_reward_training_metadata.csv"))

In [151]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "domcomp_pilot_reward_training_medpc.csv"))

In [154]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "domcomp_pilot_reward_training_medpc_metadata.csv"))